<a href="https://colab.research.google.com/github/Prince125047/job_recommendation/blob/main/Resume_parsing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install pdfplumber python-docx spacy sklearn joblib
!python -m spacy download en_core_web_sm

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 89.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [6]:
from google.colab import files

uploaded = files.upload()  # Opens a file upload dialog

Saving resume 5.pdf to resume 5.pdf


In [7]:
import os

for filename in uploaded.keys():
    print(f"Uploaded file: {filename}")
    filepath = os.path.join("/content", filename)  # File path for further processing

Uploaded file: resume 5.pdf


In [13]:
!pip install pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 55.5 MB/s eta 0:00:00


In [15]:
!pip install flashtext

  Preparing metadata (setup.py) ... done
  Created wheel for flashtext: filename=flashtext-2.7-py2.py3-none-any.whl size=9298 sha256=42ecc2cd78647f7b25bcde2e5eb4cdc54bc3b0cc6b6619e60b1e01857843398e
  Stored in directory: /root/.cache/pip/wheels/49/20/47/f03dfa8a7239c54cbc44ff7389eefbf888d2c1873edaaec888
Successfully built flashtext


In [18]:
import spacy
import fitz  # PyMuPDF for better PDF parsing
import docx
import re
import json
from flashtext import KeywordProcessor
from google.colab import files

# Load NLP Model
nlp = spacy.load("en_core_web_sm")

# Skill Extraction Using FlashText (More Accurate)
skill_keywords = [
    "python", "java", "c++", "javascript", "sql", "machine learning", "deep learning",
    "nlp", "flask", "django", "react", "node.js", "data science", "cloud computing",
    "tensorflow", "pytorch", "kubernetes", "docker", "devops", "excel", "css", "html",
    "spark", "hadoop", "big data", "power bi", "tableau", "aws", "azure", "git", "linux"
]
skill_processor = KeywordProcessor()
skill_processor.add_keywords_from_list(skill_keywords)

# Function to extract text from PDF (Better Accuracy)
def extract_text_from_pdf(pdf_path):
    text = ""
    doc = fitz.open(pdf_path)
    for page in doc:
        text += page.get_text("text") + "\n"
    return text.strip()

# Function to extract text from DOCX
def extract_text_from_docx(docx_path):
    doc = docx.Document(docx_path)
    text = "\n".join([para.text for para in doc.paragraphs])
    return text.strip()

# Function to clean extracted text (Removes unwanted data)
def clean_text(text):
    text = re.sub(r"\n+", "\n", text)  # Remove extra newlines
    text = re.sub(r"\s{2,}", " ", text)  # Remove extra spaces
    return text.strip()

# Function to extract structured information from Resume
def extract_resume_details(text):
    text = clean_text(text)
    doc = nlp(text)

    # Extract Skills using FlashText (Fast & More Accurate)
    skills = list(set(skill_processor.extract_keywords(text)))

    # Extract Education using Pattern Matching
    education_keywords = ["bachelor", "master", "phd", "b.tech", "m.tech", "b.sc", "m.sc", "degree"]
    education = [line.strip() for line in text.split("\n") if any(edu in line.lower() for edu in education_keywords)]

    # Extract Experience using Regex
    experience_match = re.findall(r"(\d+)\+?\s?(years?|yrs?)", text.lower())
    experience = experience_match[0][0] + " years" if experience_match else "Not Mentioned"

    # Extract Job Role using Named Entity Recognition (NER)
    job_roles = [ent.text for ent in doc.ents if ent.label_ in ["ORG", "JOB_TITLE", "WORK_OF_ART"]]
    job_roles = list(set([role for role in job_roles if len(role) > 2]))  # Remove short words

    # Structured Resume Data Output
    resume_data = {
        "Skills": skills if skills else ["Not Detected"],
        "Education": education if education else ["Not Detected"],
        "Experience": experience,
        "Job Role": job_roles if job_roles else ["Not Detected"]
    }

    return resume_data

# Upload Resume File
uploaded = files.upload()
file_path = list(uploaded.keys())[0]

# Extract text based on file format
if file_path.endswith(".pdf"):
    resume_text = extract_text_from_pdf(file_path)
elif file_path.endswith(".docx"):
    resume_text = extract_text_from_docx(file_path)
else:
    raise ValueError("Unsupported file format. Upload PDF or DOCX.")

# Parse Resume & Get Structured Output
parsed_resume = extract_resume_details(resume_text)

# Print Structured Resume Data
print(json.dumps(parsed_resume, indent=4))


Saving UIUX_Resume1.pdf to UIUX_Resume1.pdf
{
    "Skills": [
        "css",
        "html"
    ],
    "Education": [
        "Worked directly with clients to create brand identities and website layouts. Created print materials for Weapons of Mass Creation Fest. Made Photoshop mockups from scratch for the Arsenal web store. Mastered the pen tool."
    ],
    "Experience": "Not Mentioned",
    "Job Role": [
        "CSS & SASS",
        "Graphic Design",
        "Wordpress",
        "Stamps Leadership Scholar",
        "Arsenal",
        "Photoshop, Illustrator, InDesign, Premiere Pro",
        "miami university",
        "print & digital collateral"
    ]
}
